In [1]:
%matplotlib inline
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy.random as npr
import re
import statsmodels.api as sm

/home/peter/opt/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# this is Table S4, downloaded as a TSV.
denovo = pd.read_csv('table_s4_germline_de_novo.tsv', sep='\t', skiprows=1)  # first row is title/description

# Each de novo mutation is listed twice (once for each tissue), and we want it just once.
denovo.drop_duplicates(['individual_id', 'position'], inplace=True)
denovo['both'] = denovo.eval('heuristic == 1 and opl == 1').astype(int)
denovo.head(2)

,FID,mother_id,individual_id,tissue,position,MAF,heuristic,opl,opl_posterior_prob_de_novo,both
0,F156,F156m1,F156m1c1,bl,13951,0.05747,1,0,0.000043,0
1,F157,F157m1,F157m1c2,bl,16240,0.06672,1,0,0.344165,0


In [3]:
# Load in information on all 345 individuals
fam = pd.read_csv('famfile.tsv', sep='\t')
# Convert mother's age at birth from days to years.
fam['age_birth'] /= 365.0
print 'num tissue samples:', fam.shape[0]
print 'num individuals:', int(fam.shape[0]/2)
fam.head(2)

num tissue samples: 690
num individuals: 345


,fqid,FID,mother_id,individual_id,level,tissue,tissue_id,Sex,fam_str,fam_cat,mot_cat,age_collection,age_birth
0,TR1329M_S13,F117,F117m1,F117m1c1,c1,bl,F117m1c1_bl,Female,0-0-1-2,m1c2,NaN,6620,21.024658
1,TR1329M_S6,F117,F117m1,F117m1c1,c1,ch,F117m1c1_ch,Female,0-0-1-2,m1c2,NaN,6620,21.024658


In [4]:
# Drop duplicates across tissues since we just want the age of every indivdual.
fam.drop_duplicates(['individual_id'], inplace=True)
assert fam.shape[0] == 345

In [5]:
# Discard individuals at the top of their pedigree. fam_cat[0] == level[0]
filt = fam.apply(lambda x: x['level'][0] != x['fam_cat'][0], axis=1)
print 'number of individuals whose mothers were sequenced (345 individuals - 96 heads of family):', filt.sum()
fam_have_mother = fam.loc[filt,:]
# Make sure we have mother's age at birth for all these individuals.
assert fam_have_mother['age_birth'].isnull().sum() == 0

number of individuals whose mothers were sequenced (345 individuals - 96 heads of family): 249


In [6]:
# Make the regression data.
regdat = fam_have_mother[['individual_id', 'age_birth']].set_index('individual_id')
heur_counts = denovo.groupby('individual_id')['heuristic'].sum()
opl_counts = denovo.groupby('individual_id')['opl'].sum()
intersection_counts = denovo.groupby('individual_id')['both'].sum()
regdat['heur'] = 0
regdat['heur'] = regdat['heur'].add(heur_counts, fill_value=0).astype(int)
regdat['opl'] = 0
regdat['opl'] = regdat['opl'].add(opl_counts, fill_value=0).astype(int)
regdat['intersection'] = 0
regdat['intersection'] = regdat['intersection'].add(intersection_counts, fill_value=0).astype(int)
regdat.head(2)

,age_birth,heur,opl,intersection
individual_id,,,,
F117m1c1,21.024658,0,0,0
F294m1c1,36.673973,0,1,0


In [32]:
# Perform the Poisson regression for the OPL mutations.
fit = sm.GLM(regdat['opl'], sm.add_constant(regdat['age_birth']),
       family=sm.families.Poisson()).fit()
print 'predicted de novo mutations age 20: {:.3f}'.format(fit.predict([[1, 20]])[0])
print 'predicted de novo mutations age 40: {:.3f}'.format(fit.predict([[1, 40]])[0])
print 'difference 40-20 years of age: {:.3f}'.format(np.diff(fit.predict([[1, 20], [1,40]]))[0])
# We perform a one-sided test for an increase,
# as a decrease makes no biological sense and
# has no precedent.
print 'one-sided p-value: {:.3f}'.format(fit.pvalues['age_birth']/2.0)
fit.summary()

predicted de novo mutations age 20: 0.226
predicted de novo mutations age 40: 0.523
difference 40-20 years of age: 0.297
one-sided p-value: 0.015


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    opl   No. Observations:                  249
Model:                            GLM   Df Residuals:                      247
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -187.61
Date:                Fri, 12 Apr 2019   Deviance:                       231.80
Time:                        10:15:46   Pearson chi2:                     274.
No. Iterations:                     5                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3259      0.602     -3.864      0.000      -3.506      -1.146
age_birth      0.0419      0.019      2.159      0.031       0.004       0.080
==============================================================================
"""

In [33]:
# Perform the Poisson regression for the heuristic mutations.
fit = sm.GLM(regdat['heur'], sm.add_constant(regdat['age_birth']),
       family=sm.families.Poisson(),
       link=sm.families.links.Log()).fit()
print 'one-sided p-value: {:.3f}'.format(fit.pvalues['age_birth']/2.0)
fit.summary()

one-sided p-value: 0.087


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   heur   No. Observations:                  249
Model:                            GLM   Df Residuals:                      247
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -179.92
Date:                Fri, 12 Apr 2019   Deviance:                       226.74
Time:                        10:16:32   Pearson chi2:                     290.
No. Iterations:                     5                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9776      0.620     -3.188      0.001      -3.193      -0.762
age_birth      0.0276      0.020      1.359      0.174      -0.012       0.067
==============================================================================
"""

In [34]:
# Perform the Poisson regression for the intersection
# of the OPL and heuristic mutations.
fit = sm.GLM(regdat['intersection'], sm.add_constant(regdat['age_birth']),
       family=sm.families.Poisson()).fit()
print 'predicted de novo mutations age 20: {:.3f}'.format(fit.predict([[1, 20]])[0])
print 'predicted de novo mutations age 40: {:.3f}'.format(fit.predict([[1, 40]])[0])
print 'difference 40-20 years of age: {:.3f}'.format(np.diff(fit.predict([[1, 20], [1,40]]))[0])
print 'one-sided p-value: {:.3f}'.format(fit.pvalues['age_birth']/2.0)
fit.summary()

predicted de novo mutations age 20: 0.124
predicted de novo mutations age 40: 0.327
difference 40-20 years of age: 0.203
one-sided p-value: 0.028


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           intersection   No. Observations:                  249
Model:                            GLM   Df Residuals:                      247
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -134.68
Date:                Fri, 12 Apr 2019   Deviance:                       181.83
Time:                        10:16:49   Pearson chi2:                     286.
No. Iterations:                     6                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0585      0.790     -3.873      0.000      -4.606      -1.511
age_birth      0.0485      0.025      1.917      0.055      -0.001       0.098
==============================================================================
"""